## Streaming流式响应提升用户体验

对于 LLMs，流式处理已成为一项越来越受欢迎的功能。Streaming是在开始生成token时快速返回，而不是等待创建完整响应后再返回所有的内容，这样也可以模拟真人对话的节奏感，让用户感觉不那么"机器人"。就像你体验过的 GPT、Claude.ai、GLM等 AI 能力一样，它们在用户提问后，都采用了打字机形式的回复方式，让用户感受到数据在不断流入屏幕，从而提高了聊天的流畅性，**并极大的加快了首次回应给用户的响应的时间**，即Time to First Token (TTFT)。

1. Streaming流式响应的作用:
   - 逐个令牌(token)流式传输响应
   - 用户可以实时看到响应的生成过程
   

2. streaming流式响应的优点:
   - 避免长时间等待完整响应
   - 提高效率和交互性
   - 使对话感觉更自然,更像人类思考过程


服务端来发送streaming：


需要实时数据的场景,传统的HTTP请求-响应模型存在一些局限性——客户端主动发请求，服务端被动地返回响应的单向通信模式。这种模型在客户端需要实时获取结果的场景下是不合适的，因为这意味着客户端需要不断地轮询，所以最好的做法是服务端生成结果之后，主动推送给客户端。

可能有小伙伴知道WebSocket，WebSocket是一种双向通信协议,允许服务器和客户端之间建立持久连接,双方都可以主动发送数据，常用于实时聊天应用、在线游戏中。但选用它有些太重了，因为这意味着要从 HTTP 切换到 WebSocket 协议，以及它是双向通信的，而我们只需要接收就可以了。

有没有办法，我们仍然使用 HTTP 协议，同时还能让服务端主动推送数据呢？

铛铛铛铛～  SSE 技术来了，它的英文全称是 Server-Sent Events（服务端推送事件）。通过 SSE 可以让服务端即时推送数据到客户端，而不需要客户端轮询服务端以获取更新。

[图片1]

[图片2]


OpenAI API 的流式响应也采用 Server-Sent Events (SSE) 格式，这种基于 HTTP 的单向数据流协议。服务器会不断地发送事件数据，每个事件包含一个类型和一些数据。例如：

```
data: {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268762,"model":"gpt-4o", "choices":[{"delta":{"content":"Hello"},"index":0,"finish_reason":null}]}

data: {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268762,"model":"gpt-4o", "choices":[{"delta":{"content":", "},"index":0,"finish_reason":null}]}

data: {"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268762,"model":"gpt-4o", "choices":[{"delta":{"content":"world!"},"index":0,"finish_reason":null}]}

data: [DONE]
```

- 流式传输就像用水管接水一样，数据不是一次性全部传输完成的，而是像水流一样持续传输。这样做的优势在于：

实时性强： 可以更快地获取到第一批数据，不用等到所有数据都传输完毕。

节省内存： 不需要将所有数据都存储在内存中，可以边接收边处理。

处理大数据： 对于非常大的数据，流式传输可以避免内存溢出。

这个时候需要大量的解析和状态管理，不过不用担心，我们导入的"OpenAI"类会解析 SSE 数据流，并将其转换为 Python 对象，方便用户处理。


既然OpenAI公司已经在sdk里面帮助我们处理包装过服务器显示的原始数据转换为用户友好的python消息对象了，来方便我们处理流式响应。

对于我们来说，想要使用streaming的功能，我们一个是要开启它，一个是要解析包装过的流式响应结果，Python对象就可以了。

一共有create(stream=true),stream()两种方式，我们先看第一种：

In [2]:
from openai import OpenAI

#自动寻找 "OPENAI_API_KEY" 的环境变量
client = OpenAI()

而要从 API 获取流式响应，只需将 stream=True 传递给 client.chat.completions.create 即可。

观察在没有开启之前，响应是等LLM生成结束后一起返回给我们的：

In [6]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "为什么空调能吹出冷热两种不同的风？",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=300,
    stream=True
)
print("开始有响应了!")
for chunk in stream:
    print(chunk)

开始有响应了!
ChatCompletionChunk(id='chatcmpl-AQtjt8sgTTdxoB6HOVuEKBsy9bwpk', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974153, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1', usage=None)
ChatCompletionChunk(id='chatcmpl-AQtjt8sgTTdxoB6HOVuEKBsy9bwpk', choices=[Choice(delta=ChoiceDelta(content='空', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974153, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1', usage=None)
ChatCompletionChunk(id='chatcmpl-AQtjt8sgTTdxoB6HOVuEKBsy9bwpk', choices=[Choice(delta=ChoiceDelta(content='调', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974153, m

In [5]:
type(stream)

openai.Stream


### Streaming 的本质

经过上面的体验，我们已经体会到了streaming的魅力，就像是在看视频直播。直播不是一次性把整个视频传给你，而是源源不断地传输视频片段。Streaming API 也是这样工作的。

但它是具体怎么实现的呢，stream的本质是什么？

### 对比两种数据返回方式

### 1. 普通返回
```python
# 一次性返回所有数据
response = client.chat.completions.create(stream=False)
print(response.choices[0].message.content)
```
- 像下载完整视频才能看
- 必须等待全部数据准备好
- 占用更多内存

### 2. 流式返回(Streaming)
```python
# 陆续返回数据
response = client.chat.completions.create(stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content, end='')
```
- 像看直播一样实时显示
- 数据来一点显示一点
- 内存占用小

### 技术实现：迭代器

Streaming 用迭代器来实现这种"流动"的效果：
1. 迭代器就像一个指针，指向当前处理的数据
2. 通过 `for` 循环，迭代器不断移动并返回下一个数据
3. 处理完一个数据，前面的数据就可以释放内存了



其response返回值 `stream` 看起来是一个对象，但因为它实现了 Python 的迭代器协议（Iterator Protocol），这使得它可以被用在 for 循环中。

具体来说：
1. `Stream` 对象实现了 `__iter__()` 和 `__next__()` 方法
2. 这使得它成为一个可迭代对象（iterable）
3. 每次调用 `__next__()` 会返回流中的下一个数据块（chunk）所以我们可以看一下chunk的结构来提取内容

简单代码的话，就像这样：
```python
class Stream:
    def __iter__(self):
        return self
        
    def __next__(self):
        # 获取下一个数据块
        if has_more_data:
            return next_chunk
        else:
            raise StopIteration

# 所以可以这样使用
for chunk in stream:
    process(chunk)
```

上面讲的并不是我们主要关心的，引为OPENAI官方已经帮我们包装好了，所以对于我们应用端，接下来使用 `for chunk in stream` 这种方式来遍历它的内容就好了~

In [7]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "讲几个冷知识",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=600,
    stream=True
)
# 处理流式响应
for chunk in stream:
    print(chunk)

ChatCompletionChunk(id='chatcmpl-AQtnIrRzeCrPfzTDnJDR0cnuhde40', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974364, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1', usage=None)
ChatCompletionChunk(id='chatcmpl-AQtnIrRzeCrPfzTDnJDR0cnuhde40', choices=[Choice(delta=ChoiceDelta(content='当然', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974364, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1', usage=None)
ChatCompletionChunk(id='chatcmpl-AQtnIrRzeCrPfzTDnJDR0cnuhde40', choices=[Choice(delta=ChoiceDelta(content='可以', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730974364, model='

In [9]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "有透明的金属吗？",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=600,
    stream=True
)
# 处理流式响应
for chunk in stream:
    # 提取流式块中的内容部分
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content)

是
的
，
确
实
存在
透明
的
金
属
。如
氮
化
铝
（
Al
uminum
 Nit
ride
,
 Al
N
）
和
氧
化
铝
（
Al
uminum
 Ox
ide
,
 Al
₂
O
₃
）
等
材料
，
虽然
它
们
是
金
属
氧
化
物
，但
在
某
些
情况下
可以
表现
出
透明
性
。此外
，
近年来
，
科学
家
们
开发
了一
些
新
型
的
材料
，例如
某
些
合
金
或
特殊
处理
的
金
属
涂
层
，可以
在
特
定
波
长
下
呈
现
出
透明
性
。


然而
，
传统
意义
上的
金
属
通常
是不
透明
的
，因为
它
们
的
电子
结构
会
阻
止
光
的
透
过
。因此
，
透明
金
属
的
研究
仍
然
是
一个
前
沿
领域
，
持续
吸
引
着
材料
科学
和
光
学
的
关注
。


在 Python 中，`print()` 函数的默认行为是输出内容写入缓冲区，然后再从缓冲区中输出到终端。并在行末自动添加一个换行符（`'\n'`）。

所以为了在输出逐渐到达时保持实时更新，并且将输出显示在同一行，以便更清晰地查看完整的响应，在处理实时流数据时常特定设置 `flush=True` 和 `end=""`。


1. **`end=""`**:
   - 这个参数定义了 `print()` 语句输出的结尾字符。默认情况下是换行符 `'\n'`，但当你设置 `end=""` 时，输出内容将不会自动换行，而是会在同一行继续输出。
   - **应用场景**：在处理流数据时，可能需要将所有数据连续打印在同一行，而不是逐个事件打印时都换行。

2. **`flush=True`**(可选):
   - `print()` 函数将输出内容写入缓冲区，然后再从缓冲区中输出到终端。`flush` 参数控制是否立即将缓冲区中的内容刷新到终端。
   - `flush=False`（默认）：`print()` 函数会根据系统的设置决定何时刷新缓冲区，有时输出可能会被延迟。
   - **`flush=True`**：强制 `print()` 立即将缓冲区的内容输出到终端。这对于希望实时看到输出的情况非常有用，尤其是在流式数据处理中，用户通常希望尽快看到新的数据。


In [10]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "讲几个冷知识",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=600,
    stream=True
)
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end="", flush=True)

当然可以！以下是几个有趣的冷知识：

1. **香蕉是浆果**：在植物学上，香蕉被归类为浆果，而草莓则不是。这是因为草莓的种子在果实的外部，而真正的浆果的种子则在果实内部。

2. **水草可以“呼吸”**：水草如水葫芦和沉水植物实际上会通过光合作用释放氧气，并在水中形成气泡，这些气泡可以帮助小鱼和昆虫更容易地获取氧气。

3. **齿轮在蜗牛的壳中**：某些海洋蜗牛的壳中有齿轮状的结构，有助于它们更有效地移动。这些结构在生物机械学中被认为非常独特。

4. **地球上的水量几乎是固定的**：地球上的水不会消失或增加，实际上，水循环在本质上是一个封闭系统。地球上的水量在数十亿年间保持相对稳定。

5. **记忆的多样性**：人类的大脑对于记忆的储存和回忆非常复杂，许多记忆并不是“重播”过去的情景，而是由多个线索重构而成的，这也就是为什么不同的人对于同一事件的记忆会有所不同。

希望这些冷知识能给你带来一些新鲜的见解！

如果我们想要访问有关令牌使用情况的信息，则有两种方法，一种是自己用tiktoken估算（历史开发者遗留，不推荐），一种是用官方的参数设置生成usage（默认是没有usage信息）：

第一种：

In [24]:
import openai
import tiktoken


# 选择模型的编码方式
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 初始化令牌计数器
token_count = 0

# 创建流式聊天完成请求
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "根据热力学第二定律，世界将越来越混乱。那为什么会产生能体现秩序的细胞、生物和人类？"}],
    stream=True
)

# 处理流式响应
for chunk in response:
    # 提取流式块中的内容部分
    if chunk.choices[0].delta.content:
        content_chunk =chunk.choices[0].delta.content
        # 计算当前内容块的令牌数量并累加
        token_count += len(encoding.encode(content_chunk))
        # 实时输出内容
        print(content_chunk, end="")

# 输出总的令牌使用数量
print(f"\n总共使用的令牌数量（估计值）：{token_count}")


根据热力学第二定律，孤立系统的熵是不断增加的，意味着系统的无序程度会随着时间而增加。然而，这并不意味着在开放系统中秩序无法形成。细胞、生物和人类的出现可以通过以下几个方面来理解：

1. **开放系统**：地球并不是一个孤立系统，而是一个开放系统。它从太阳获得能量，利用这部分能量来维持生命和组织结构。生物体通过吸收能量，以低熵的形式（如光能、化学能）来维持其有序结构。

2. **局部熵降低**：生物体的生长和发展是局部熵降低的结果。在摄取和利用能量的过程中，生物体虽然降低了自身的熵，但整体宇宙的熵却在增加。例如，植物通过光合作用将阳光转化为化学能，并形成复杂的有机分子。

3. **自组织**：生物界中的许多结构和过程可以通过自组织原理来解释。生物分子在特定条件下能够自发地组装成复杂的结构，如蛋白质、细胞膜等，这些过程本身是有序的，体现了自然界中的秩序形成机制。

4. **进化**：生物进化是一个通过自然选择促使有序结构和功能增强的过程。那些能够更好地适应环境的生物，往往会生存下来并繁衍后代，从而在基因层面上形成更复杂、有序的生命形式。

总之，虽然热力学第二定律在宏观上描述了无序程度的增加，但在有能量输入和适当条件下，局部系统（如生物体）可以通过自组织和能量转换形成秩序。
总共使用的令牌数量（估计值）：413


第二种，首先要添加stream_options={"include_usage": True}，api文档：https://platform.openai.com/docs/api-reference/chat/create#chat-create-stream_options：

In [30]:
import openai
import tiktoken


# 选择模型的编码方式
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 初始化令牌计数器
token_count = 0

# 创建流式聊天完成请求
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "根据热力学第二定律，世界将越来越混乱。那为什么会产生能体现秩序的细胞、生物和人类？"}],
    stream=True,
    stream_options={"include_usage": True}
)

for chunk in response:
    print(chunk)


ChatCompletionChunk(id='chatcmpl-AQn4tqIeOMhE1TO676REny4LXZr5Z', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730948547, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_9b78b61c52', usage=None)
ChatCompletionChunk(id='chatcmpl-AQn4tqIeOMhE1TO676REny4LXZr5Z', choices=[Choice(delta=ChoiceDelta(content='热', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730948547, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_9b78b61c52', usage=None)
ChatCompletionChunk(id='chatcmpl-AQn4tqIeOMhE1TO676REny4LXZr5Z', choices=[Choice(delta=ChoiceDelta(content='力', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1730948547, model='gp

In [33]:
import openai
import tiktoken


# 选择模型的编码方式
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 初始化令牌计数器
token_count = 0

# 创建流式聊天完成请求
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "根据热力学第二定律，世界将越来越混乱。那为什么会产生能体现秩序的细胞、生物和人类？"}],
    stream=True,
    stream_options={"include_usage": True}
)

for chunk in response:
    if chunk.choices and chunk.choices[0].delta.content:  # 检查 choices 非空且有内容
        content_chunk = chunk.choices[0].delta.content
        print(content_chunk, end='', flush=True)
    if chunk.usage:
        usage = chunk.usage
        prompt_tokens = usage.prompt_tokens
        completion_tokens = usage.completion_tokens
        total_tokens = usage.total_tokens
        print(f"\nPrompt Tokens: {prompt_tokens}")
        print(f"Completion Tokens: {completion_tokens}")
        print(f"Total Tokens: {total_tokens}")


热力学第二定律确实指出，在一个封闭系统中，熵（混乱程度）会随时间增加。然而，这并不意味着在所有系统中秩序不可能产生。实际上，熵增的概念主要适用于孤立系统，而生物体和生态系统并不是孤立的。以下是一些解释如何在熵增的背景下仍然出现秩序的观点：

1. **开放系统**：生物体是开放系统，它们与外界进行能量和物质的交换。通过摄取能量（如食物、阳光）并将其转化为有序的结构和功能，生物体可以局部降低熵。虽然生物体内部的熵可能降低，但整个环境的熵却相应增加，从而遵循热力学第二定律。

2. **能量的输入**：生物体通过代谢过程不断吸收和转换能量，这些能量用于维持细胞结构、进行生理活动和繁殖等。通过这种能量的输入，可以维持和创造有序的生命形式。

3. **自组织现象**：在某些条件下，系统可以自发地形成有序结构。例如，细胞在发育过程中会通过基因表达和信号传导等机制自我组织成复杂的结构和功能。这样的自组织现象在自然界中普遍存在，它们依赖于局部的能量流和信息传递。

4. **进化机制**：生物通过自然选择和进化过程适应环境，这一过程也促进了有序结构的形成。适应环境的个体能够生存和繁殖，从而在遗传上传递出有序的生物特征。

总之，生物体和生态系统的有序性是通过与外界的能量和物质交换、局部自组织现象以及进化适应等过程实现的。这并不违反热力学第二定律，而是展示了在熵增的总体背景下如何能够局部地降低熵，形成复杂的生命现象。
Prompt Tokens: 39
Completion Tokens: 467
Total Tokens: 506


中间被max token截止仍然有usage chunk：

In [11]:
import openai
import tiktoken


# 选择模型的编码方式
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 初始化令牌计数器
token_count = 0

# 创建流式聊天完成请求
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "根据热力学第二定律，世界将越来越混乱。那为什么会产生能体现秩序的细胞、生物和人类？"}],
    stream=True,
    stream_options={"include_usage": True},
    max_tokens=10
    
)

for chunk in response:
    if chunk.choices and chunk.choices[0].delta.content:  # 检查 choices 非空且有内容
        content_chunk = chunk.choices[0].delta.content
        print(content_chunk, end='', flush=True)
    if chunk.usage:
        usage = chunk.usage
        prompt_tokens = usage.prompt_tokens
        completion_tokens = usage.completion_tokens
        total_tokens = usage.total_tokens
        print(f"\nPrompt Tokens: {prompt_tokens}")
        print(f"Completion Tokens: {completion_tokens}")
        print(f"Total Tokens: {total_tokens}")


热力学第二定律确实指出，在
Prompt Tokens: 39
Completion Tokens: 10
Total Tokens: 49


流式处理现在已经为我们所用了！并大大地提升了用户体验，但并不会神奇地提高 model 生成响应所需的总时间。在页面上展示首token的速度是快了多，但从请求开始到收到最终生成的token之间仍然需要相同的时间。

1. 回顾Time to First Token (TTFT) 定义与重要性：
   - TTFT是从发送请求到接收到AI模型生成的第一个内容单元所需的时间。
   - 它是衡量AI响应速度和用户体验的关键指标，对于提供快速、流畅的交互至关重要。

2. 流式传输对TTFT的改善：
   - 流式传输允许内容一旦生成就立即发送，而不是等待整个响应完成。
   - 这显著减少了用户的等待时间，提升了交互体验，特别适用于聊天机器人、长文本生成和实时翻译等场景。

我们用代码来看看时间方面的结果：

非流式处理方法：

In [12]:
import time
def measure_non_streaming_ttft():
    start_time = time.time()

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "写一篇长篇文章，解释唐朝的历史",
            }
        ],
        temperature=0,
        model="gpt-4o-mini",
    )

    response_time = time.time() - start_time

    print(response.choices[0].message.content)
    print(f"接收第一个token的时间: {response_time:.3f}秒")
    print(f"接收完整响应的时间: {response_time:.3f}秒")
    print(f"生成的总token数: {response.usage.total_tokens}")

流式处理方法：

In [13]:
import time
def measure_streaming_ttft():
    start_time = time.time()

    stream = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "写一篇长篇文章，解释唐朝的历史",
            }
        ],
        temperature=0,
        model="gpt-4o-mini",
        stream_options={"include_usage": True},
        stream=True
    )
    
    have_received_first_token = False
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:  # 检查 choices 非空且有内容
            if not have_received_first_token:
                ttft = time.time() - start_time
                have_received_first_token = True
            content_chunk = chunk.choices[0].delta.content
            print(content_chunk, end='', flush=True)
        if chunk.usage:
            usage = chunk.usage     
            total_tokens = usage.total_tokens

 
    total_time = time.time() - start_time
    print(f"\n接收第一个token的时间: {ttft:.3f}秒", flush=True)
    print(f"接收完整响应的时间: {total_time:.3f}秒", flush=True)
    print(f"生成的总token数: {total_tokens}", flush=True)

In [14]:
measure_non_streaming_ttft()

唐朝（618年－907年）是中国历史上一个辉煌的朝代，通常被认为是中国封建社会的巅峰时期之一。唐朝的建立、发展、繁荣以及衰落，都是中国历史上重要的篇章，影响深远。

### 一、唐朝的建立

唐朝的建立可以追溯到隋朝末年。隋朝在短短的几十年内统一了中国，但由于统治者的暴政和连年的战争，导致民众苦不堪言，社会动荡不安。公元617年，李渊在太原起兵，反抗隋朝的统治。经过几年的战争，李渊于618年攻入长安，建立了唐朝，年号“武德”，李渊即位为唐高祖。

### 二、唐朝的盛世

唐朝的盛世主要体现在经济、文化和政治等多个方面。

#### 1. 经济繁荣

唐朝时期，农业、手工业和商业都得到了长足的发展。政府实施了一系列的土地改革政策，如均田制，确保农民的基本生活需求。同时，唐朝的丝绸之路也得到了发展，促进了与中亚、西亚及欧洲的贸易往来，带来了大量的财富。

#### 2. 文化辉煌

唐朝是中国古代文化的高峰期之一，文学、艺术、宗教等领域都取得了显著成就。唐诗是这一时期最为突出的文化成就，涌现出如李白、杜甫、王维等一大批伟大的诗人。唐朝的绘画、书法、音乐和舞蹈等艺术形式也达到了极高的水平。

#### 3. 政治制度

唐朝建立了较为完善的中央集权制度，设立了三省六部制，增强了中央政府的权威。同时，唐朝还重视科举制度的建设，通过考试选拔人才，打破了门阀世家的垄断，使得更多平民有机会进入仕途。

### 三、对外交流

唐朝时期是中国与外部世界交流最为频繁的时期之一。长安作为当时的国际大都市，吸引了大量的外国商人、使节和文化交流者。佛教在这一时期得到了广泛传播，许多寺庙和佛教艺术品在唐朝时期得以兴盛。

### 四、唐朝的衰落

尽管唐朝在初期取得了辉煌的成就，但随着时间的推移，内部的腐败和外部的压力逐渐显现，导致了唐朝的衰落。

#### 1. 内部腐败

随着唐朝的繁荣，官僚体系逐渐腐败，贪污腐败现象严重，导致民众对统治者的不满加剧。此外，藩镇割据现象严重，地方势力逐渐强大，中央政府的控制力减弱。

#### 2. 外部压力

唐朝后期，外部的威胁也日益增加。尤其是来自于契丹、吐蕃等少数民族的侵扰，使得唐朝的国力受到严重削弱。公元755年，安史之乱爆发，给唐朝带来了巨大的灾难，导致了大量的人口死亡和经济的崩溃。

### 五、唐朝的灭亡

907年，唐朝最终灭亡，标志

In [15]:
measure_streaming_ttft()

唐朝（618年－907年）是中国历史上一个辉煌的朝代，通常被认为是中国封建社会的巅峰时期之一。唐朝的建立、发展、繁荣以及衰落，都是中国历史上重要的篇章。以下将从唐朝的建立、政治制度、经济文化、对外关系以及衰落等方面进行详细阐述。

### 一、唐朝的建立

唐朝的建立可以追溯到隋朝末年。隋朝在短短的几十年内实现了统一，但由于统治者的暴政和连年的战争，导致民众苦不堪言，最终引发了大规模的农民起义。公元617年，李渊在太原起兵，经过几年的征战，最终在618年攻入长安，建立了唐朝，年号“武德”。李渊即位为唐高祖，开始了唐朝的统治。

### 二、政治制度

唐朝的政治制度在中国历史上具有重要的影响。唐朝建立了较为完善的中央集权制度，设立了三省六部制。三省分别是中书省、门下省和尚书省，负责不同的政务。六部则是吏部、户部、礼部、兵部、刑部和工部，负责具体的行政事务。这一制度在后来的朝代中被广泛借鉴。

唐朝还实行科举制度，选拔人才。科举制度的建立，使得更多的平民有机会通过考试进入仕途，打破了贵族对官职的垄断，促进了社会的流动性。

### 三、经济文化

唐朝是中国历史上经济繁荣、文化昌盛的时期。农业方面，唐朝推广了良种和先进的耕作技术，粮食产量大幅提高。手工业和商业也得到了迅速发展，长安、洛阳等城市成为当时世界上最大的城市之一，商业繁荣，市井文化兴盛。

文化方面，唐朝是中国古代文学的高峰期，诗歌、散文、小说等各类文学形式都取得了显著成就。唐诗尤其闻名于世，李白、杜甫、王维等诗人创作了大量脍炙人口的诗篇，至今仍被广泛传颂。此外，唐朝的绘画、书法、音乐、舞蹈等艺术形式也达到了很高的水平。

### 四、对外关系

唐朝时期，中国与周边国家的交流频繁，形成了“丝绸之路”的繁荣局面。唐朝的国力强盛，吸引了大量的外国使节、商人和文化交流者。唐朝与日本、朝鲜、东南亚等国建立了良好的外交关系，促进了文化的传播和经济的交流。

唐朝还积极派遣使节和军队进行对外扩张，曾经征服了西域的多个国家，开辟了更为广阔的疆域。唐朝的文化、宗教（如佛教）也通过这些交流传播到其他国家，影响深远。

### 五、衰落与灭亡

唐朝的衰落始于8世纪中叶，主要原因包括政治腐败、宦官专权、藩镇割据等。安史之乱（755年－763年）是唐朝历史上的重大事件，导致了国家的严重动荡。虽然唐朝在安史之乱后经过一段时间的恢复

优化：使用`with` 语句进行**资源管理**

**stream的本质是迭代器生成器，并涉及了资源使用（网络连接），应该被正确的关闭**

`with` 语句是 Python 中一种强大的语法结构，用于简化资源管理，确保资源在使用完毕后能够被正确释放。它主要用于处理文件、网络连接、数据库连接等需要手动关闭的资源，避免资源泄漏和程序错误。

**`with` 语句的基本语法：**

1. 首先,我们要明白两个关键概念:
   - `with` 语句是一个语法结构
   - 上下文管理器是一个实现了 `__enter__()` 和 `__exit__()` 方法的对象

2. 来看一个具体例子:
```python
class MyContext:
    def __enter__(self):
        print("进入上下文")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("退出上下文")

# 使用 with 语句
with MyContext() as mc:
    print("在上下文中")
```

3. 执行流程是这样的:
   1. `with` 语句检查 MyContext() 是否是一个合法的上下文管理器(是否实现了所需方法)
   2. 调用上下文管理器对象的 `__enter__()` 方法
   3. 执行 `with` 语句块中的代码
   4. 最后调用 `__exit__()` 方法


- "使用 `with` 语句时,所管理的上下文管理器对象会自动调用其 `__enter__()` 和 `__exit__()` 方法"

就像开门关门:
- `with` 语句就像是一个使用门的规范流程
- 而开门(`__enter__`)和关门(`__exit__`)的具体动作是由门(上下文管理器)来实现的

**示例：**

**1. 文件操作：**

```python
with open('myfile.txt', 'r') as f:
    contents = f.read()
    # 处理文件内容
# 文件会自动关闭，无需手动调用 f.close()
```

**2. 数据库连接：**

```python
with connect('mydb.sqlite') as conn:
    cursor = conn.cursor()
    # 执行数据库操作
# 数据库连接会自动关闭
```

`with` 语句确保无论代码块中发生什么（包括异常），`stream` 对象都会被正确关闭。这意味着与 API 的连接会被及时释放，避免资源泄漏。它更安全可靠，能够保证资源的正确释放，避免潜在的资源泄漏问题。尤其是 **处理大型流式响应，程序可能出现异常，编写健壮的代码** 场景下，使用 `with` 语句更加重要，实际开发中应始终优先选择使用 `with` 语句来管理流式响应对象，养成良好的编程习惯。


In [39]:
with client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "讲几个德国的有趣的知识",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=400,
    temperature=0,
    stream=True,
) as stream:
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, flush=True, end="")


当然可以！以下是一些关于德国的有趣知识：

1. **啤酒文化**：德国是世界著名的啤酒生产国，尤其以巴伐利亚州的啤酒闻名。每年十月，慕尼黑会举办世界上最大的啤酒节——“慕尼黑啤酒节”（Oktoberfest），吸引数百万游客。

2. **汽车工业**：德国是汽车工业的发源地之一，许多知名汽车品牌如大众、宝马、奔驰和保时捷均在德国成立。德国的高速公路（Autobahn）上有些路段没有速度限制，吸引了许多汽车爱好者。

3. **环保意识**：德国在环保和可再生能源方面走在世界前列。德国政府积极推动可再生能源的使用，尤其是风能和太阳能，力求在2030年前实现更高比例的可再生能源使用。

4. **节日和传统**：德国有许多独特的节日和传统，比如圣诞市场（Weihnachtsmarkt），每年冬季在各大城市举行，出售手工艺品、圣诞装饰和美食。

5. **语言和方言**：德语是德国的官方语言，但在不同地区有许多方言。例如，巴伐利亚方言和高地德语（Hochdeutsch）之间的差异很大，甚至有时会让人难以理解。

6. **教育系统**：德国的教育系统以其高质量和多样性而闻名。德国的大学通常不收取学费，吸引了大量国际学生前来学习。

7. **历史遗迹**：德国有许多历史悠久的城堡和遗迹，如新天鹅堡

如果不用with，手动管理连接：

In [ ]:
stream = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "有透明的金属吗？",
        }
    ],
    model="gpt-4o-mini",
    max_tokens=400,
    temperature=0,
    stream=True,
)
try:
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, flush=True, end="")
finally:#是在 try 块结束时执行的，即使中途发生异常也会触发，确保连接被正确关闭，以避免资源泄漏或未关闭连接。
    stream.close()  # 手动关闭连接
    
print("Exiting...")

## 如何在tool use 中启用流式传输

In [17]:
import json
from typing import Dict, Any

def get_weather(city):
    return {
        "city": city,
        "date": "明天",
        "temperature": 22,
        "weather": "小雨",
        "humidity": 85,
        "wind": "东北风3级"
    }
# 定义tools配置
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

启用流式：

In [19]:
messages=[{"role": "user", "content": "北京今天天气如何？"}]
with client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    stream=True
)as stream:
    for chunk in stream:
        print(chunk)

ChatCompletionChunk(id='chatcmpl-AQuA3NMkP9NzCvGbFOgbXMos9GnU8', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=[ChoiceDeltaToolCall(index=0, id='call_9uuyXdn5PsVHKRMKumBvqSga', function=ChoiceDeltaToolCallFunction(arguments='', name='get_weather'), type='function')]), finish_reason=None, index=0, logprobs=None)], created=1730975775, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1', usage=None)
ChatCompletionChunk(id='chatcmpl-AQuA3NMkP9NzCvGbFOgbXMos9GnU8', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='{"', name=None), type=None)]), finish_reason=None, index=0, logprobs=None)], created=1730975775, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_0ba0d124f1

In [ ]:
#不启用流式的解析
# tool_call = response.choices[0].message.tool_calls[0].function
# print(response.choices[0].message)
# print(tool_call.name)
# print(tool_call.arguments)

In [51]:
import json

messages = [{"role": "user", "content": "北京今天天气如何？"}]

with client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    stream=True
) as stream:
    function_call_content = ''
    function_name = ''
    for chunk in stream:
        # 检查是否有函数调用的信息
        if chunk.choices[0].delta.tool_calls:
            print(chunk.choices[0].delta.tool_calls)

[ChoiceDeltaToolCall(index=0, id='call_kKj0CYnxye9vehXj0ixSE33X', function=ChoiceDeltaToolCallFunction(arguments='', name='get_weather'), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='{"', name=None), type=None)]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='city', name=None), type=None)]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='":"', name=None), type=None)]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='北京', name=None), type=None)]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='"}', name=None), type=None)]


分析这些数据流好做进一步提取：

1. 第一条消息包含了工具调用的基本信息——唯一的调用ID,要调用的函数名
2. 后续消息是函数参数的流式传输，逐步构建JSON字符串——要调用的函数参数：
```python
{"city":"北京"}
```

- 工具调用信息是分开传输的：
   - 首先传递调用的元信息（ID、函数名等）
   - 然后分片传输函数参数，需要把所有参数片段拼接才能得到完整参数，形成一个有效的JSON对象



In [57]:
import json

messages = [{"role": "user", "content": "北京今天天气如何？"}]

with client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    stream=True
) as stream:
    function_call_content = ''
    function_name = ''
    for chunk in stream:
        # 检查是否有函数调用的信息
        if chunk.choices[0].delta.tool_calls:
            for tool_call in chunk.choices[0].delta.tool_calls:
                # 获取函数名
                if tool_call.function.name:
                    function_name = tool_call.function.name
                # 拼接函数参数
                if tool_call.function.arguments:
                    function_call_content += tool_call.function.arguments

    print(function_name)
    print(function_call_content)

get_weather
{"city":"北京"}


In [21]:
import json

messages = [ {"role": "system", "content": "根据天气信息，请你根据现实准确专业的给出出行，穿衣，活动建议"},
             {"role": "user", "content": "北京今天天气如何？"}]

with client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    stream=True
) as stream:
    function_call_content = ''
    function_name = ''
    for chunk in stream:
        # 检查是否有函数调用的信息
        if chunk.choices[0].delta.tool_calls:
            for tool_call in chunk.choices[0].delta.tool_calls:
                # 获取函数名
                if tool_call.function.name:
                    function_name = tool_call.function.name
                # 拼接函数参数
                if tool_call.function.arguments:
                    function_call_content += tool_call.function.arguments
        # 检查是否完成了函数调用
        if chunk.choices[0].finish_reason == 'tool_calls':
            # 解析函数参数
            try:
                args = json.loads(function_call_content)
                
            except json.JSONDecodeError:
                print("解析JSON时出错")
                args = {}
            # 调用函数
            if function_name == 'get_weather':
                result = get_weather(**args)
                # 将函数调用信息添加到 messages
                messages.append({
                    "role": "assistant",
                    "content": None,
                    "function_call": {
                        "name": function_name,
                        "arguments": function_call_content
                    }
                })
                # 将函数结果添加到 messages
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": json.dumps(result)
                })
                # 获取助手的最终回复
                final_response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    stream=True
                )
                for chunk in final_response:
                    if chunk.choices[0].delta.content:
                        print(chunk.choices[0].delta.content,end='')
            break


根据天气信息，北京明天的天气预报如下：

- **气温**：22°C
- **天气**：小雨
- **湿度**：85%
- **风**：东北风3级

### 出行建议：
- **出行**：建议带上一把雨伞，以防小雨影响出行。
- **交通**：由于下雨天气，请注意路面湿滑，行车时保持安全距离，减速慢行。
  
### 穿衣建议：
- **服装**：可以穿轻便的长袖衬衫或者薄外套，搭配防水的鞋子，以保持干燥。
- **配件**：可选择穿戴透气的防水外套，保持一定的温暖，同时避免雨水直接打湿。

### 活动建议：
- **户外活动**：如果要进行户外活动，最好选择覆盖的场所，或参观室内场馆。
- **室内活动**：可以考虑去咖啡厅、图书馆或商场等室内场所放松，也可进行一些室内运动。

总体来看，明天的小雨天气，出行和活动安排上建议以灵活应对为主。